In [2]:
import os
import os.path
import datetime as dt
import gzip
import json

import numpy as np
import pandas as pd



# **EXTRACTION**

Las siguientes líneas de código permitirán acceder a los archivos comprimidos .json.gz descargados que se encuentran en la carpeta [PI MLOps - STEAM](https://drive.google.com/drive/folders/1HqBG2-sUkz_R3h1dZU5F2uAzpRn7BSpj)

Estos archivos tiene archivos JSON anidados, por lo que no se puede leer directamente con `pd.read_json()`. Así que, creamos una función `read_multiple_json()` que lee un archivo comprimido json.gzip anidado tomando como parámetro la ruta del archivo

In [3]:
def read_multiple_json(ruta_archivo):

    # creamos una lista 'data' que almacenará cada componete del json
    data = []
    with gzip.open(ruta_archivo, 'rt', encoding='utf-8') as file:
        for line in file:
            data.append(line)
    
    # Iteramos a través de la lista 'data' y convertimos en diccionarios
    # la función eval() convierte cada cadena en un diccionario y se almacena en la lista 'resultados'
    resultados = []
    for cadena in data:
        diccionario = eval(cadena)
        resultados.append(diccionario)
    
    # convertimos la lista 'resultados' en un dataframe, tomando como columna las llaves del primer diccionario de la lista
    df = pd.DataFrame(resultados, columns= resultados[0].keys())
    return df

Construimos los dataframes

In [4]:
df_reviews = read_multiple_json('..\\STEAM-MLOps\\Dataset\\user_reviews.json.gz')

In [5]:
df_items = read_multiple_json('..\\STEAM-MLOps\\Dataset\\users_items.json.gz')

Si intentamos leer el archivo `steam_games` con el método anterior nos retornará error; ya que el método **eval()** no puede leer valores NaN que no están definidos en python. Así que utilizamos la función **json.load()** en su lugar para manejar los valores NaN automáticamente.

In [6]:
def read_multiple_json_nan(ruta_archivo):

    # creamos una lista 'data' que almacenará cada componete del json
    data = []
    with gzip.open(ruta_archivo, 'rt', encoding='utf-8') as file:
        for line in file:
            data.append(line)
    
    # Iteramos a través de la lista 'data' y convertimos en diccionarios
    # la función json.loads() convierte cada cadena en un diccionario y se almacena en la lista 'resultados'
    resultados = []
    for cadena in data:
        try:
            diccionario = json.loads(cadena)
        except ValueError:
            diccionario = None
        resultados.append(diccionario)
    
    # convertimos la lista 'resultados' en un dataframe, tomando como columna las llaves del primer diccionario de la lista
    df = pd.DataFrame(resultados, columns= resultados[0].keys())
    return df

Abrimos el archivo `steam_games.json`

In [7]:
steam_games = read_multiple_json_nan('..\\STEAM-MLOps\\Dataset\\steam_games.json.gz')


# **TRANSFORM**

Se llevará a cabo la transformación y limpieza de los datos en cada Dataframe de forma individual (corrección de errores en los datos, transformación de los datos en un formato adecuado para un análisis posterior, etc). Una vez que se han limpiado y transformado los datos de cada uno, se procederá a unir los Dataframes resultantes a través de una columna en común.

## Tratamiento de **`steam_games`**

In [8]:
steam_games.head(3)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
steam_games.tail(2)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"
120444,NaN,NaN,Maze Run VR,NaN,http://store.steampowered.com/app/681550/Maze_...,NaN,"[Early Access, Adventure, Indie, Action, Simul...",http://steamcommunity.com/app/681550/reviews/?...,"[Single-player, Stats, Steam Leaderboards, HTC...",4.99,True,681550,NaN


Eliminamos columnas inncesarias para análisis posteriores

In [11]:
steam_games = steam_games[['id', 'title', 'genres', 'release_date', 'tags', 'specs']]
steam_games

,id,title,genres,release_date,tags,specs
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
120440,773640,Colony On Mars,"[Casual, Indie, Simulation, Strategy]",2018-01-04,"[Strategy, Indie, Casual, Simulation]","[Single-player, Steam Achievements]"
120441,733530,LOGistICAL: South Africa,"[Casual, Indie, Strategy]",2018-01-04,"[Strategy, Indie, Casual]","[Single-player, Steam Achievements, Steam Clou..."
120442,610660,Russian Roads,"[Indie, Racing, Simulation]",2018-01-04,"[Indie, Simulation, Racing]","[Single-player, Steam Achievements, Steam Trad..."
120443,658870,EXIT 2 - Directions,"[Casual, Indie]",2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...","[Single-player, Steam Achievements, Steam Cloud]"


Eliminamos valores nulos del nuevo dataframe **games**

In [12]:
steam_games.dropna(inplace=True)
steam_games

,id,title,genres,release_date,tags,specs
0,761140,Lost Summoner Kitty,"[Action, Casual, Indie, Simulation, Strategy]",2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player]
1,643980,Ironbound,"[Free to Play, Indie, RPG, Strategy]",2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla..."
2,670290,Real Pool 3D - Poolians,"[Casual, Free to Play, Indie, Simulation, Sports]",2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla..."
3,767400,弹炸人2222,"[Action, Adventure, Casual]",2017-12-07,"[Action, Adventure, Casual]",[Single-player]
4,772540,Battle Royale Trainer,"[Action, Adventure, Simulation]",2018-01-04,"[Action, Adventure, Simulation, FPS, Shooter, ...","[Single-player, Steam Achievements]"
...,...,...,...,...,...,...
28519,745400,Kebab it Up!,"[Action, Adventure, Casual, Indie]",2018-01-04,"[Action, Indie, Casual, Violent, Adventure]","[Single-player, Steam Achievements, Steam Cloud]"
28520,773640,Colony On Mars,"[Casual, Indie, Simulation, Strategy]",2018-01-04,"[Strategy, Indie, Casual, Simulation]","[Single-player, Steam Achievements]"
28521,733530,LOGistICAL: South Africa,"[Casual, Indie, Strategy]",2018-01-04,"[Strategy, Indie, Casual]","[Single-player, Steam Achievements, Steam Clou..."
28522,610660,Russian Roads,"[Indie, Racing, Simulation]",2018-01-04,"[Indie, Simulation, Racing]","[Single-player, Steam Achievements, Steam Trad..."


Cambiamos el nombre de la columna **id** a **item_id**

In [13]:
steam_games.rename(columns={'id':'item_id'}, inplace=True)

Eliminamos datos duplicados y Cambiamos los tipos de datos

In [14]:
steam_games.drop_duplicates(subset='item_id', inplace=True)
steam_games

,item_id,title,genres,release_date,tags,specs
0,761140,Lost Summoner Kitty,"[Action, Casual, Indie, Simulation, Strategy]",2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player]
1,643980,Ironbound,"[Free to Play, Indie, RPG, Strategy]",2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla..."
2,670290,Real Pool 3D - Poolians,"[Casual, Free to Play, Indie, Simulation, Sports]",2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla..."
3,767400,弹炸人2222,"[Action, Adventure, Casual]",2017-12-07,"[Action, Adventure, Casual]",[Single-player]
4,772540,Battle Royale Trainer,"[Action, Adventure, Simulation]",2018-01-04,"[Action, Adventure, Simulation, FPS, Shooter, ...","[Single-player, Steam Achievements]"
...,...,...,...,...,...,...
28519,745400,Kebab it Up!,"[Action, Adventure, Casual, Indie]",2018-01-04,"[Action, Indie, Casual, Violent, Adventure]","[Single-player, Steam Achievements, Steam Cloud]"
28520,773640,Colony On Mars,"[Casual, Indie, Simulation, Strategy]",2018-01-04,"[Strategy, Indie, Casual, Simulation]","[Single-player, Steam Achievements]"
28521,733530,LOGistICAL: South Africa,"[Casual, Indie, Strategy]",2018-01-04,"[Strategy, Indie, Casual]","[Single-player, Steam Achievements, Steam Clou..."
28522,610660,Russian Roads,"[Indie, Racing, Simulation]",2018-01-04,"[Indie, Simulation, Racing]","[Single-player, Steam Achievements, Steam Trad..."


In [15]:
# Información sobre las columnas
steam_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28523 entries, 0 to 28523
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   item_id       28523 non-null  object
 1   title         28523 non-null  object
 2   genres        28523 non-null  object
 3   release_date  28523 non-null  object
 4   tags          28523 non-null  object
 5   specs         28523 non-null  object
dtypes: object(6)
memory usage: 1.5+ MB


Cambiamos los tipos de datos

In [21]:
# Columna 'item_id' lo cambiamos a tipo numérico
# Columna 'release_date' lo cambiamos a tipo numérico y los errores en las fechas se convierten en nulos
steam_games['item_id'] = pd.to_numeric(steam_games['item_id'])
steam_games['release_date'] = pd.to_datetime(steam_games['release_date'], errors='coerce')

# Eliminamos valores nulos
steam_games.dropna(inplace=True)

#reindexamos
steam_games.reset_index(drop=True, inplace=True)
steam_games

,item_id,title,genres,release_date,tags,specs
0,761140,Lost Summoner Kitty,"[Action, Casual, Indie, Simulation, Strategy]",2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player]
1,643980,Ironbound,"[Free to Play, Indie, RPG, Strategy]",2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla..."
2,670290,Real Pool 3D - Poolians,"[Casual, Free to Play, Indie, Simulation, Sports]",2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla..."
3,767400,弹炸人2222,"[Action, Adventure, Casual]",2017-12-07,"[Action, Adventure, Casual]",[Single-player]
4,772540,Battle Royale Trainer,"[Action, Adventure, Simulation]",2018-01-04,"[Action, Adventure, Simulation, FPS, Shooter, ...","[Single-player, Steam Achievements]"
...,...,...,...,...,...,...
28233,745400,Kebab it Up!,"[Action, Adventure, Casual, Indie]",2018-01-04,"[Action, Indie, Casual, Violent, Adventure]","[Single-player, Steam Achievements, Steam Cloud]"
28234,773640,Colony On Mars,"[Casual, Indie, Simulation, Strategy]",2018-01-04,"[Strategy, Indie, Casual, Simulation]","[Single-player, Steam Achievements]"
28235,733530,LOGistICAL: South Africa,"[Casual, Indie, Strategy]",2018-01-04,"[Strategy, Indie, Casual]","[Single-player, Steam Achievements, Steam Clou..."
28236,610660,Russian Roads,"[Indie, Racing, Simulation]",2018-01-04,"[Indie, Simulation, Racing]","[Single-player, Steam Achievements, Steam Trad..."


In [26]:
steam_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28238 entries, 0 to 28237
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   item_id       28238 non-null  int64         
 1   title         28238 non-null  object        
 2   genres        28238 non-null  object        
 3   release_date  28238 non-null  datetime64[ns]
 4   tags          28238 non-null  object        
 5   specs         28238 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 1.3+ MB


### Tomamos columnas útiles para la API en un dataframe `games`

Para optimizar procesos en el desarrollo de la API, tomaremos tres columnas del dataframe **steam_games**:

- **item_id**: id del juego
- **title**: nombre del juego
- **genres**: géneros del juego
- **release_date**: año de lanzamiento

In [22]:
games = steam_games[['item_id', 'title', 'genres', 'release_date']]
games

,item_id,title,genres,release_date
0,761140,Lost Summoner Kitty,"[Action, Casual, Indie, Simulation, Strategy]",2018-01-04
1,643980,Ironbound,"[Free to Play, Indie, RPG, Strategy]",2018-01-04
2,670290,Real Pool 3D - Poolians,"[Casual, Free to Play, Indie, Simulation, Sports]",2017-07-24
3,767400,弹炸人2222,"[Action, Adventure, Casual]",2017-12-07
4,772540,Battle Royale Trainer,"[Action, Adventure, Simulation]",2018-01-04
...,...,...,...,...
28233,745400,Kebab it Up!,"[Action, Adventure, Casual, Indie]",2018-01-04
28234,773640,Colony On Mars,"[Casual, Indie, Simulation, Strategy]",2018-01-04
28235,733530,LOGistICAL: South Africa,"[Casual, Indie, Strategy]",2018-01-04
28236,610660,Russian Roads,"[Indie, Racing, Simulation]",2018-01-04


Descomponemos los archivos de tipo lista de `genres`

In [23]:
games = games.explode('genres')
games.reset_index(drop=True, inplace=True)
games

,item_id,title,genres,release_date
0,761140,Lost Summoner Kitty,Action,2018-01-04
1,761140,Lost Summoner Kitty,Casual,2018-01-04
2,761140,Lost Summoner Kitty,Indie,2018-01-04
3,761140,Lost Summoner Kitty,Simulation,2018-01-04
4,761140,Lost Summoner Kitty,Strategy,2018-01-04
...,...,...,...,...
70093,610660,Russian Roads,Indie,2018-01-04
70094,610660,Russian Roads,Racing,2018-01-04
70095,610660,Russian Roads,Simulation,2018-01-04
70096,658870,EXIT 2 - Directions,Casual,2017-09-02


Agregamos una columna `year` en la que tomamos en año de la columna `release_date`

In [24]:
games['year_release'] = games['release_date'].dt.year
games

,item_id,title,genres,release_date,year_release
0,761140,Lost Summoner Kitty,Action,2018-01-04,2018
1,761140,Lost Summoner Kitty,Casual,2018-01-04,2018
2,761140,Lost Summoner Kitty,Indie,2018-01-04,2018
3,761140,Lost Summoner Kitty,Simulation,2018-01-04,2018
4,761140,Lost Summoner Kitty,Strategy,2018-01-04,2018
...,...,...,...,...,...
70093,610660,Russian Roads,Indie,2018-01-04,2018
70094,610660,Russian Roads,Racing,2018-01-04,2018
70095,610660,Russian Roads,Simulation,2018-01-04,2018
70096,658870,EXIT 2 - Directions,Casual,2017-09-02,2017


In [25]:
# Mostramos la información de la tabla 'games'
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70098 entries, 0 to 70097
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   item_id       70098 non-null  int64         
 1   title         70098 non-null  object        
 2   genres        70098 non-null  object        
 3   release_date  70098 non-null  datetime64[ns]
 4   year_release  70098 non-null  int32         
dtypes: datetime64[ns](1), int32(1), int64(1), object(2)
memory usage: 2.4+ MB


## **`user_reviews`**

In [53]:
df_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [54]:
# eliminamos la columna url
user_reviews = df_reviews[['user_id', 'reviews']]
user_reviews

,user_id,reviews
0,76561197970982479,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...
25794,76561198306599751,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


Extraemos los archivos JSON de la columna **reviews**

In [55]:
# Creamos una lista vacía para almacenar las filas resultantes
nuevas_filas = []

# Iteramos a través de cada fila del DataFrame
for index, row in user_reviews.iterrows():
    user_id = row['user_id']    # los valores de la columna 'user_id' se guardan en la variable user_id
    lista_reviews = row['reviews']  # los diccionarios de la columna 'reviews' se almacenan como listas en la variable lista_reviews
    
    # Iteramos a través de la columna 'reviews'
    for diccionario in lista_reviews:
        nueva_fila = {'user_id': user_id, **diccionario}    # se crea un diccionario 'nueva_fila' que combina el 'user_id' con el diccionario desempaquetado
        nuevas_filas.append(nueva_fila)     # llenamos la lista vacía inicial con los diccionarios 'nueva_fila'

# Creamos un nuevo DataFrame desplegado con la lista de diccionarios 'nuevas_filas'
user_reviews_desplegado = pd.DataFrame(nuevas_filas)

In [56]:
user_reviews_desplegado.head()

,user_id,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


Convertimos la columna `recommend` de booleano a entero

In [57]:
user_reviews_desplegado['recommend'] = user_reviews_desplegado['recommend'].astype(int)

Eliminamos el texto "Posted" de la columna `posted`

In [58]:
user_reviews_desplegado['posted'] = user_reviews_desplegado['posted'].str.replace('Posted ', '')
user_reviews_desplegado['posted'] = user_reviews_desplegado['posted'].str.replace('.', '')

Limpiamos la columna `posted`: nos quedamos solo con las que contengan fecha completa (mes, día y año).

**Desición**: Si miramos con detenimiento el archivo JSON; las filas que contienen fechas completas se distinguen por tener una coma (,) que separa el año del mes y día. Así que, nos quedamos con las filas que contengan comas.

In [59]:
user_reviews_desplegado = user_reviews_desplegado[user_reviews_desplegado['posted'].str.contains(',')]

Convertimos `item_id` a entero y `posted` a Date

In [60]:
user_reviews_desplegado['posted'] = pd.to_datetime(user_reviews_desplegado['posted'], format='%B %d, %Y')
user_reviews_desplegado['item_id'] = pd.to_numeric(user_reviews_desplegado['item_id'])
user_reviews_desplegado

,user_id,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,,2011-11-05,,1250,No ratings yet,1,Simple yet with great replayability. In my opi...
1,76561197970982479,,2011-07-15,,22200,No ratings yet,1,It's unique and worth a playthrough.
2,76561197970982479,,2011-04-21,,43110,No ratings yet,1,Great atmosphere. The gunplay can be a bit chu...
3,js41637,,2014-06-24,,251610,15 of 20 people (75%) found this review helpful,1,I know what you think when you see this title ...
4,js41637,,2013-09-08,,227300,0 of 1 people (0%) found this review helpful,1,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...,...
59252,wayfeng,1 person found this review funny,2015-10-14,,730,1 of 1 people (100%) found this review helpful,1,its FUNNNNNNNN
59255,76561198251004808,,2015-10-10,,253980,No ratings yet,1,Awesome fantasy game if you don't mind the gra...
59265,72947282842,,2015-10-31,,730,No ratings yet,1,Prettyy Mad Game
59267,ApxLGhost,,2015-12-14,,730,No ratings yet,1,AMAZING GAME 10/10


Eliminamos columnas innecesarias

In [61]:
user_reviews_desplegado.drop(columns=['funny', 'last_edited', 'helpful'], inplace=True)

In [62]:
# reindexamos
user_reviews_desplegado.reset_index(drop=True, inplace=True)
user_reviews_desplegado

,user_id,posted,item_id,recommend,review
0,76561197970982479,2011-11-05,1250,1,Simple yet with great replayability. In my opi...
1,76561197970982479,2011-07-15,22200,1,It's unique and worth a playthrough.
2,76561197970982479,2011-04-21,43110,1,Great atmosphere. The gunplay can be a bit chu...
3,js41637,2014-06-24,251610,1,I know what you think when you see this title ...
4,js41637,2013-09-08,227300,1,For a simple (it's actually not all that simpl...
...,...,...,...,...,...
49181,wayfeng,2015-10-14,730,1,its FUNNNNNNNN
49182,76561198251004808,2015-10-10,253980,1,Awesome fantasy game if you don't mind the gra...
49183,72947282842,2015-10-31,730,1,Prettyy Mad Game
49184,ApxLGhost,2015-12-14,730,1,AMAZING GAME 10/10


Eliminamos datos duplicados

In [63]:
user_reviews_desplegado.drop_duplicates(inplace=True)
user_reviews_desplegado

,user_id,posted,item_id,recommend,review
0,76561197970982479,2011-11-05,1250,1,Simple yet with great replayability. In my opi...
1,76561197970982479,2011-07-15,22200,1,It's unique and worth a playthrough.
2,76561197970982479,2011-04-21,43110,1,Great atmosphere. The gunplay can be a bit chu...
3,js41637,2014-06-24,251610,1,I know what you think when you see this title ...
4,js41637,2013-09-08,227300,1,For a simple (it's actually not all that simpl...
...,...,...,...,...,...
49181,wayfeng,2015-10-14,730,1,its FUNNNNNNNN
49182,76561198251004808,2015-10-10,253980,1,Awesome fantasy game if you don't mind the gra...
49183,72947282842,2015-10-31,730,1,Prettyy Mad Game
49184,ApxLGhost,2015-12-14,730,1,AMAZING GAME 10/10


### Columna **sentiment_analysis**
creamos la columna `sentiment_analysis` aplicando NLP sobre la columna **review** que tendrá la siguiente escala:
- 0: malo
- 1: neutral
- 2: bueno

Utilizamos el packpage **Sentiment** de **NLTK**.<br>
**Sentiment** necesita el archivo `lexicon` para utilizar sus métodos

In [ ]:
nltk.download('vader_lexicon')

In [64]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# instanciamos un objeto que analizará el sentimiento de los review
analizador = SentimentIntensityAnalyzer()
'''
    creamos la columna 'polarity' aplicando el método polarity_scores sobre la columna review
    el método 'polarity_scores' devuelve un valor flotante que representa la fuerza del sentimiento basado en el texto de entrada
    polarity < 0 ---> sentimiento negativo
    polarity = 0 ---> sentimiento neutro
    polarity > 0 ---> sentimiento positivo
'''
user_reviews_desplegado['polarity'] = user_reviews_desplegado['review'].apply(lambda r: analizador.polarity_scores(r)['compound'])

In [65]:
user_reviews_desplegado

,user_id,posted,item_id,recommend,review,polarity
0,76561197970982479,2011-11-05,1250,1,Simple yet with great replayability. In my opi...,0.8481
1,76561197970982479,2011-07-15,22200,1,It's unique and worth a playthrough.,0.2263
2,76561197970982479,2011-04-21,43110,1,Great atmosphere. The gunplay can be a bit chu...,0.9117
3,js41637,2014-06-24,251610,1,I know what you think when you see this title ...,0.9566
4,js41637,2013-09-08,227300,1,For a simple (it's actually not all that simpl...,0.9708
...,...,...,...,...,...,...
49181,wayfeng,2015-10-14,730,1,its FUNNNNNNNN,0.0000
49182,76561198251004808,2015-10-10,253980,1,Awesome fantasy game if you don't mind the gra...,0.7650
49183,72947282842,2015-10-31,730,1,Prettyy Mad Game,-0.4939
49184,ApxLGhost,2015-12-14,730,1,AMAZING GAME 10/10,0.6739


En base al peso asignado en la columna `polarity`, construimos la columna **`sentiment_analysis`**

In [66]:
def sentimiento(peso):
    if peso<0: # sentimiento negativo
        return 0
    elif peso>0: # sentimiento neutro
        return 2
    else: # sentimiento positivo
        return 1
    
user_reviews_desplegado.loc[:,'sentiment_analysis'] = user_reviews_desplegado['polarity'].apply(sentimiento)

In [67]:
user_reviews_desplegado

,user_id,posted,item_id,recommend,review,polarity,sentiment_analysis
0,76561197970982479,2011-11-05,1250,1,Simple yet with great replayability. In my opi...,0.8481,2
1,76561197970982479,2011-07-15,22200,1,It's unique and worth a playthrough.,0.2263,2
2,76561197970982479,2011-04-21,43110,1,Great atmosphere. The gunplay can be a bit chu...,0.9117,2
3,js41637,2014-06-24,251610,1,I know what you think when you see this title ...,0.9566,2
4,js41637,2013-09-08,227300,1,For a simple (it's actually not all that simpl...,0.9708,2
...,...,...,...,...,...,...,...
49181,wayfeng,2015-10-14,730,1,its FUNNNNNNNN,0.0000,1
49182,76561198251004808,2015-10-10,253980,1,Awesome fantasy game if you don't mind the gra...,0.7650,2
49183,72947282842,2015-10-31,730,1,Prettyy Mad Game,-0.4939,0
49184,ApxLGhost,2015-12-14,730,1,AMAZING GAME 10/10,0.6739,2


Eliminamos las columnas `review` y `polarity`

In [68]:
user_reviews_desplegado.drop(columns=['review', 'polarity'], inplace=True)
user_reviews_desplegado

,user_id,posted,item_id,recommend,sentiment_analysis
0,76561197970982479,2011-11-05,1250,1,2
1,76561197970982479,2011-07-15,22200,1,2
2,76561197970982479,2011-04-21,43110,1,2
3,js41637,2014-06-24,251610,1,2
4,js41637,2013-09-08,227300,1,2
...,...,...,...,...,...
49181,wayfeng,2015-10-14,730,1,1
49182,76561198251004808,2015-10-10,253980,1,2
49183,72947282842,2015-10-31,730,1,0
49184,ApxLGhost,2015-12-14,730,1,2


In [69]:
user_reviews_desplegado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48498 entries, 0 to 49185
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   user_id             48498 non-null  object        
 1   posted              48498 non-null  datetime64[ns]
 2   item_id             48498 non-null  int64         
 3   recommend           48498 non-null  int32         
 4   sentiment_analysis  48498 non-null  int64         
dtypes: datetime64[ns](1), int32(1), int64(2), object(1)
memory usage: 2.0+ MB


Eliminamos espacios de más en las columnas de tipo str

In [70]:
user_reviews_desplegado['user_id'] = user_reviews_desplegado['user_id'].str.strip()
user_reviews_desplegado

,user_id,posted,item_id,recommend,sentiment_analysis
0,76561197970982479,2011-11-05,1250,1,2
1,76561197970982479,2011-07-15,22200,1,2
2,76561197970982479,2011-04-21,43110,1,2
3,js41637,2014-06-24,251610,1,2
4,js41637,2013-09-08,227300,1,2
...,...,...,...,...,...
49181,wayfeng,2015-10-14,730,1,1
49182,76561198251004808,2015-10-10,253980,1,2
49183,72947282842,2015-10-31,730,1,0
49184,ApxLGhost,2015-12-14,730,1,2


Creamos una columna `year_posted` que toma el año de la columna `posted`

In [71]:
user_reviews_desplegado['year_posted'] = user_reviews_desplegado['posted'].dt.year
user_reviews_desplegado

,user_id,posted,item_id,recommend,sentiment_analysis,year_posted
0,76561197970982479,2011-11-05,1250,1,2,2011
1,76561197970982479,2011-07-15,22200,1,2,2011
2,76561197970982479,2011-04-21,43110,1,2,2011
3,js41637,2014-06-24,251610,1,2,2014
4,js41637,2013-09-08,227300,1,2,2013
...,...,...,...,...,...,...
49181,wayfeng,2015-10-14,730,1,1,2015
49182,76561198251004808,2015-10-10,253980,1,2,2015
49183,72947282842,2015-10-31,730,1,0,2015
49184,ApxLGhost,2015-12-14,730,1,2,2015


In [72]:
# Mostramos la información de la tabla 'user_review'
user_reviews_desplegado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48498 entries, 0 to 49185
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   user_id             48498 non-null  object        
 1   posted              48498 non-null  datetime64[ns]
 2   item_id             48498 non-null  int64         
 3   recommend           48498 non-null  int32         
 4   sentiment_analysis  48498 non-null  int64         
 5   year_posted         48498 non-null  int32         
dtypes: datetime64[ns](1), int32(2), int64(2), object(1)
memory usage: 2.2+ MB


## **`user_items`**

In [73]:
df_items

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


Tomamos las columnas necesarias

In [74]:
user_items = df_items[['user_id', 'items']]
user_items

,user_id,items
0,76561197970982479,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...
88305,76561198323066619,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,[]
88308,76561198329548331,"[{'item_id': '304930', 'item_name': 'Unturned'..."


Extraemos los archivos JSON de la columna **items**

In [75]:
# Creamos una lista vacía para almacenar las filas resultantes
nuevas_filas = []

# Iteramos a través de las filas del DataFrame
for index, row in user_items.iterrows():
    user_id = row['user_id']
    lista_reviews = row['items']
    
    # Iteramos a través de la lista de reseñas
    for diccionario in lista_reviews:
        nueva_fila = {'user_id': user_id, **diccionario}
        nuevas_filas.append(nueva_fila)

# Creamos un nuevo DataFrame desplegado con las filas resultantes
user_items_desplegado = pd.DataFrame(nuevas_filas)

In [76]:
user_items_desplegado

,user_id,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,10,Counter-Strike,6,0
1,76561197970982479,20,Team Fortress Classic,0,0
2,76561197970982479,30,Day of Defeat,7,0
3,76561197970982479,40,Deathmatch Classic,0,0
4,76561197970982479,50,Half-Life: Opposing Force,0,0
...,...,...,...,...,...
5153204,76561198329548331,346330,BrainBread 2,0,0
5153205,76561198329548331,373330,All Is Dust,0,0
5153206,76561198329548331,388490,One Way To Die: Steam Edition,3,3
5153207,76561198329548331,521570,You Have 10 Seconds 2,4,4


Eliminamos la columna **playtime_2weeks**

In [77]:
user_items_desplegado.drop(columns='playtime_2weeks', inplace=True)

Eliminamos datos duplicados y nulos

In [78]:
user_items_desplegado.drop_duplicates(inplace=True)
user_items_desplegado.dropna(inplace=True)
user_items_desplegado

,user_id,item_id,item_name,playtime_forever
0,76561197970982479,10,Counter-Strike,6
1,76561197970982479,20,Team Fortress Classic,0
2,76561197970982479,30,Day of Defeat,7
3,76561197970982479,40,Deathmatch Classic,0
4,76561197970982479,50,Half-Life: Opposing Force,0
...,...,...,...,...
5153204,76561198329548331,346330,BrainBread 2,0
5153205,76561198329548331,373330,All Is Dust,0
5153206,76561198329548331,388490,One Way To Die: Steam Edition,3
5153207,76561198329548331,521570,You Have 10 Seconds 2,4


Convertimos `item_id` a entero y Cambiamos el nombre de la columna `item_name` a `title`

In [79]:
user_items_desplegado['item_id'] = pd.to_numeric(user_items_desplegado['item_id'])
user_items_desplegado.rename(columns={'item_name':'title'}, inplace= True)

Reindexamos

In [80]:
user_items_desplegado.reset_index(drop=True, inplace=True)
user_items_desplegado

,user_id,item_id,title,playtime_forever
0,76561197970982479,10,Counter-Strike,6
1,76561197970982479,20,Team Fortress Classic,0
2,76561197970982479,30,Day of Defeat,7
3,76561197970982479,40,Deathmatch Classic,0
4,76561197970982479,50,Half-Life: Opposing Force,0
...,...,...,...,...
5094087,76561198329548331,346330,BrainBread 2,0
5094088,76561198329548331,373330,All Is Dust,0
5094089,76561198329548331,388490,One Way To Die: Steam Edition,3
5094090,76561198329548331,521570,You Have 10 Seconds 2,4


### Buscamos outliers

**Desición**

Para eliminar outliers sin necesidad de mutilar la tabla, utilizamos la **puntuación Z (Z-score)** sobre la columna 'playtime_forever'

In [81]:
avg = user_items_desplegado['playtime_forever'].mean() # promedio del tiempo de juego
std = user_items_desplegado['playtime_forever'].std() # desviación estandar
(avg - std, avg + std)

(-4423.595507720673, 6404.624290867896)

**Limpiamos la tabla de outliers**

Según la definición de Z-score, los valores que están fuera del intervalo `(avg - std, avg + std)` se consideran outliers. Nos quedamos con los datos en los que el tiempo de juego(playtime_dorever) esté dentro de dicho intervalo

In [82]:
user_items_desplegado = user_items_desplegado[user_items_desplegado['playtime_forever']<avg + std]
user_items_desplegado

,user_id,item_id,title,playtime_forever
0,76561197970982479,10,Counter-Strike,6
1,76561197970982479,20,Team Fortress Classic,0
2,76561197970982479,30,Day of Defeat,7
3,76561197970982479,40,Deathmatch Classic,0
4,76561197970982479,50,Half-Life: Opposing Force,0
...,...,...,...,...
5094087,76561198329548331,346330,BrainBread 2,0
5094088,76561198329548331,373330,All Is Dust,0
5094089,76561198329548331,388490,One Way To Die: Steam Edition,3
5094090,76561198329548331,521570,You Have 10 Seconds 2,4


## **`merge`**

Creamos los dataframes `items_games` y `review_games` para análisis posteriores

In [83]:
# items_games será el merge entre la tabla de items y la tabla games sobre las columnas en común 'item_id' y 'title'
items_games = pd.merge(user_items_desplegado, games, how='inner', on=['item_id', 'title'])
items_games

,user_id,item_id,title,playtime_forever,genres,release_date,year_release
0,76561197970982479,10,Counter-Strike,6,Action,2000-11-01,2000
1,js41637,10,Counter-Strike,0,Action,2000-11-01,2000
2,Riot-Punch,10,Counter-Strike,0,Action,2000-11-01,2000
3,doctr,10,Counter-Strike,93,Action,2000-11-01,2000
4,corrupted_soul,10,Counter-Strike,108,Action,2000-11-01,2000
...,...,...,...,...,...,...,...
7461478,76561198071808318,444770,Mimpi Dreams,46,Indie,2016-03-22,2016
7461479,76561198107283457,354280,ChaosTower,164,Action,2016-02-23,2016
7461480,76561198107283457,354280,ChaosTower,164,Casual,2016-02-23,2016
7461481,76561198107283457,354280,ChaosTower,164,Indie,2016-02-23,2016


In [84]:
# review_games será el merge entre la tabla de reviews y la tabla games sobre la columna en común 'item_id'
reviews_games = pd.merge(user_reviews_desplegado, games, how='inner', on='item_id')
reviews_games

,user_id,posted,item_id,recommend,sentiment_analysis,year_posted,title,genres,release_date,year_release
0,76561197970982479,2011-11-05,1250,1,2,2011,Killing Floor,Action,2009-05-14,2009
1,death-hunter,2015-03-30,1250,1,2,2015,Killing Floor,Action,2009-05-14,2009
2,DJKamBer,2013-07-12,1250,1,0,2013,Killing Floor,Action,2009-05-14,2009
3,diego9031,2015-08-13,1250,1,1,2015,Killing Floor,Action,2009-05-14,2009
4,76561198081962345,2014-04-05,1250,1,1,2014,Killing Floor,Action,2009-05-14,2009
...,...,...,...,...,...,...,...,...,...,...
101582,Gamer0009,2015-08-11,306040,1,2,2015,The Howler,Simulation,2016-04-21,2016
101583,Gamer0009,2015-08-11,306040,1,2,2015,The Howler,Strategy,2016-04-21,2016
101584,llDracuwulf,2015-10-29,307130,1,2,2015,Asteria,Action,2014-07-03,2014
101585,llDracuwulf,2015-10-29,307130,1,2,2015,Asteria,Adventure,2014-07-03,2014


De la tabla `items_games` cogemos solo las columnas que usaremos para la API en un dataframe `items_games_util`

In [85]:
items_games_util = items_games[['genres', 'year_release', 'playtime_forever', 'user_id']]
items_games_util

,genres,year_release,playtime_forever,user_id
0,Action,2000,6,76561197970982479
1,Action,2000,0,js41637
2,Action,2000,0,Riot-Punch
3,Action,2000,93,doctr
4,Action,2000,108,corrupted_soul
...,...,...,...,...
7461478,Indie,2016,46,76561198071808318
7461479,Action,2016,164,76561198107283457
7461480,Casual,2016,164,76561198107283457
7461481,Indie,2016,164,76561198107283457


# **LOAD**

Exportamos las tablas `items_games` y `review_games` para posteriores análisis. Debido al gran tamaño de éstas, lo exportamos en archivos `.parquet` utilizando la biblioteca **`pyarrow`**

Instalamos la biblioteca faltante

In [155]:
pip install pyarrow

Exportamos las tablas a la carpeta 'Dataset'

In [86]:
# Guardamos el dataframe mergeado de items y games en un archivo .parquet llamado 'items_games.parquet'
items_games.to_parquet('items_games.parquet', index=False)

In [87]:
# Guardamos el dataframe que usaremos para la API en un archivo .parquet llamado 'items_games.parquet'
items_games_util.to_parquet('items_games_util.parquet', index=False)

In [88]:
# Guardamos el dataframe mergeado de reviews y games en un archivo .parquet llamado 'reviews_games.parquet'
reviews_games.to_parquet('reviews_games.parquet', index=False)

Las tablas originales desplegadas las guardamos en la carpeta **Dataset**

In [89]:
# Guardamos el datafram original 'steam_games' como un parquet
steam_games.to_parquet(os.path.join('Dataset', 'steam_games.parquet'), index=False)

In [90]:
# Guardamos el DF original 'user_items_desplegado'
user_items_desplegado.to_parquet(os.path.join('Dataset', 'user_items.parquet'), index=False)

In [91]:
# Guardamos el DF original 'user_reviews_desplegado'
user_reviews_desplegado.to_parquet(os.path.join('Dataset', 'user_reviews.parquet'), index=False)